## Code-101 (As identified in the Flowchart in the Thesis)

#### About This Document
This document is part of the coursework for the Dissertation Module (CASA0004/0010) at UCL CASA. You can find the source code [here](https://github.com/burhanucl2023/) and the data associated [here](https://github.com/burhanucl2023/)

### Purpose of this Code

This is the second in a series of four codes aimed at thoroughly analyzing the impact of the PMGSY road construction program in India on rural development, using nighttime light data as an indicator of economic activity. This document focuses on generating a dataset crucial for the econometric analysis.

**1. Data Used in this Code:**
   - **Harmonized Nighttime Light Dataset**: This dataset integrates Road Data, Nighttime Light Data, Terrain & Elevation Data, and Shrid Data, compiled through data extraction, merging, and wrangling processes in Code-100, Code-101, and Code-102.
   - **Shrid Data**: Sourced from the SHRUG dataset provided by the Development Data Lab.

**2. Objectives of this Code:**
   - Calculate the Mean Nighttime Light Value for each shrid from 1995 to 2020.

**3. Results:**
   - Produce a comprehensive nighttime light dataset, which will be used in the subsequent analysis of the PMGSY road construction program's impact in Code-103.
   - This dataset will be essential for descriptive, statistical, and econometric analysis.

The following Google Earth Engine (GEE) code is used to calculate and export the mean nighttime light values for each shrid (village) in India from 1995 to 2020. This data will be used to analyze the impact of the PMGSY road construction program on rural development. This document details the steps and code used, ensuring the analysis is transparent and reproducible.

```javascript
// Define the night lights dataset
var nightLightsList = [
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_1992_calDMSP").set('year', 1992),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_1993_calDMSP").set('year', 1993),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_1994_calDMSP").set('year', 1994),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_1995_calDMSP").set('year', 1995),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_1996_calDMSP").set('year', 1996),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_1997_calDMSP").set('year', 1997),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_1998_calDMSP").set('year', 1998),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_1999_calDMSP").set('year', 1999),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2000_calDMSP").set('year', 2000),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2001_calDMSP").set('year', 2001),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2002_calDMSP").set('year', 2002),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2003_calDMSP").set('year', 2003),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2004_calDMSP").set('year', 2004),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2005_calDMSP").set('year', 2005),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2006_calDMSP").set('year', 2006),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2007_calDMSP").set('year', 2007),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2008_calDMSP").set('year', 2008),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2009_calDMSP").set('year', 2009),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2010_calDMSP").set('year', 2010),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2011_calDMSP").set('year', 2011),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2012_calDMSP").set('year', 2012),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2013_calDMSP").set('year', 2013),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2014_simVIIRS").set('year', 2014),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2015_simVIIRS").set('year', 2015),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2016_simVIIRS").set('year', 2016),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2017_simVIIRS").set('year', 2017),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2018_simVIIRS").set('year', 2018),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2019_simVIIRS").set('year', 2019),
  ee.Image("projects/ee-burhanahwani/assets/Harmonized_DN_NTL_2020_simVIIRS").set('year', 2020)
];

var nightLights = ee.ImageCollection(nightLightsList);

// Load the shrid boundary
var shrid = ee.FeatureCollection("projects/ee-burhanahwani/assets/shrid");

// Center the map at India
Map.setCenter(78.9629, 20.5937, 4.5); // Longitude, Latitude, and Zoom level
Map.setOptions('SATELLITE');

// Visualization parameters for night lights
var visParams = {
  min: 0,
  max: 60,
  palette: ['000000', 'ff00ff', 'ffff00', 'ffffff'] // Updated color palette
};

// Function to create text annotation images
function createTextImage(text, position, scale) {
  var textFeature = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point(position), {label: text})
  ]);

  return ee.Image().paint({
    featureCollection: textFeature,
    color: 1,
    width: scale
  }).visualize({palette: ['white'], opacity: 0.01});
}

// Function to create an annotated image with the year label and make outside boundary transparent
var annotateImage = function(image, year) {
  // Clip the image to the shrid boundary
  var clippedImage = image.clip(shrid).visualize(visParams);

  // Create text annotation images
  var yearTextImage = createTextImage(year, [74.8, 34.1], 30);

  // Add boundary
  /*
  var boundary = ee.Image().byte().paint({
    featureCollection: shrid,
    color: 1,
    width: 0
  }).visualize({palette: ['white'], opacity: 1});
  */

  // Blend all layers
  return ee.ImageCollection([clippedImage, yearTextImage]).mosaic();
};

// Create a collection of annotated night light images
var annotatedImages = nightLights.map(function(image) {
  var year = ee.String(image.get('year'));
  return annotateImage(image, year).set('label', year);
});

// Create UI panel for year selection
var panel = ui.Panel({
  style: {width: '300px'}
});
ui.root.insert(0, panel);

// Convert year values to strings for UI selection
var yearOptions = nightLights.aggregate_array('year').map(function(year) {
  return ee.String(year).cat('');
}).getInfo();

var selectYear = ui.Select({
  items: yearOptions,
  onChange: function(year) {
    var image = nightLights.filter(ee.Filter.eq('year', parseInt(year))).first();
    var annotatedImage = annotateImage(image, year);
    Map.layers().set(1, ui.Map.Layer(annotatedImage, {}, 'Night Light ' + year));
  },
  style: {stretch: 'horizontal'}
});
panel.add(ui.Label('Select Year'));
panel.add(selectYear);

// Initialize the map with the first year
selectYear.setValue('1992');

// Function to calculate mean night light values for each feature in shrid
var calculateMeanNightLights = function(image) {
  return shrid.map(function(feature) {
    var meanDict = image.reduceRegion({
      reducer: ee.Reducer.mean(),
      geometry: feature.geometry(),
      scale: 927.6624195666301,
      maxPixels: 1e9
    });

    var meanValue = ee.Algorithms.If(
      meanDict.contains('b1'),
      ee.Number(meanDict.get('b1')),
      null
    );

    return feature.set({
      year: image.get('year'),
      mean: meanValue
    });
  });
};

// Calculate mean night light values for each year and export
yearOptions.forEach(function(year) {
  var image = nightLights.filter(ee.Filter.eq('year', parseInt(year))).first();
  var meanNightLights = calculateMeanNightLights(image);
  
  Export.table.toDrive({
    collection: meanNightLights,
    description: 'MeanNightLightValues_India_' + year,
    fileFormat: 'CSV',
    selectors: ['shrid2', 'year', 'mean']
  });
});
